In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def getData():
    data = pd.read_csv('../data/data.csv')

    getAllData = False
    if getAllData:
        #read all 5 csvs files in data named data FIGHTER_STAT UFC_PCA UFC_processed UFC_TEST UFC_TRAIN
        data = pd.read_csv('../data/data.csv')
        data1 = pd.read_csv('../data/UFC_PCA.csv')
        data2 = pd.read_csv('../data/UFC_processed.csv')
        data3 = pd.read_csv('../data/UFC_TEST.csv')
        data4 = pd.read_csv('../data/UFC_TRAIN.csv')
        data5 = pd.read_csv("../data/FIGHTER_STAT.csv")
    return data

In [16]:
data = getData()

In [4]:
def findWinnerByFighterName(name) -> str:
    filterR = [col for col in data if col.startswith('R')]
    filterB = [col for col in data if col.startswith('B')]
    x = data.Winner[data.R_fighter == name]
    y = data.Winner[data.B_fighter == name]
    return x, y

In [5]:
def normalizeData(data) -> pd.DataFrame:
    """
    Normalize data to be between 0 and 1
    """
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [6]:
def getNormalizedDataFrameForModels(dataFrame) -> pd.DataFrame:
    """
    Removes the columns that are not needed for the models
    Returns the dataframe normalized
    """
    
    #make a copy of data without the columns [ R_fighter B_fighter Referee date location]
    dataImportant = dataFrame.drop(['R_fighter', 'B_fighter', 'Referee', 'date', 'location'], axis=1)
    dataImportant.dropna(inplace=True)

    dataImportant = dataImportant.apply(lambda x: normalizeData(x) if x.dtype == 'int64' else x)
    dataImportant = dataImportant.apply(lambda x: normalizeData(x) if x.dtype == 'float64' else x)

    return dataImportant

In [7]:
dataImpNorm = getNormalizedDataFrameForModels(data)

In [8]:
def replaceValues(data):
    #replace Red with 0, blue with 1 and draw with 2
    data.Winner = data.Winner.replace(['Red', 'Blue', 'Draw'], [0, 1, 2])
    #replace Orthodox Southpaw Switch "Open Stance" with 0 1 2 and 3
    data.R_Stance = data.R_Stance.replace(['Open Stance', 'Switch', 'Orthodox', 'Southpaw'], [0, 1, 2, 3])
    data.B_Stance = data.B_Stance.replace(['Open Stance', 'Switch', 'Orthodox', 'Southpaw'], [0, 1, 2, 3])
    #replace True with 1 and False with 0
    data.title_bout = data.title_bout.astype(int)
    return data

In [9]:
dataImpNormClean = replaceValues(dataImpNorm)

In [10]:
def getDictOfValues(data) -> dict:
    """
    Returns a dictionary of the values of the column
    """
    uniqueDict = {}
    cont = 0
    for i in data:
        if i not in uniqueDict:
            uniqueDict[i] = cont
            cont += 1
    return uniqueDict

def replaceValuesWithDict(data) -> pd.DataFrame:
    """
    Replaces the values of a column with the values of the dictionary
    """
    uniqueDict = getDictOfValues(data)

    for i in uniqueDict:
        try:
            data[data == i] = uniqueDict[i]
        except Exception as e:
            print(e)
            
    return data


In [11]:
# this function takes the column and creates a dictionary with the values and the index, then replaces the values with the index
dataImpNormClean.weight_class = replaceValuesWithDict(dataImpNormClean.weight_class)
dataImpNormClean.weight_class = dataImpNormClean.weight_class.astype(float)

C:\Users\Lucas\AppData\Local\Temp/ipykernel_20100/4069535618.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data == i] = uniqueDict[i]


In [12]:
def getCleanDF():
    return dataImpNormClean